In [ ]:
# @title 1. Environment Initialization
from google.colab import drive
import os

if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')

print("--- Installing Toolchain ---")
!wget -O - https://apt.llvm.org/llvm-snapshot.gpg.key | sudo apt-key add -
!echo "deb http://apt.llvm.org/jammy/ llvm-toolchain-jammy-18 main" | sudo tee /etc/apt/sources.list.d/llvm.list
!apt-get update -y
!apt-get install -y libmlir-18-dev mlir-18-tools llvm-18-dev cmake ninja-build clang-18

os.environ["CC"] = "clang-18"
os.environ["CXX"] = "clang++-18"

In [ ]:
# @title 2. Project Activation
import os
import shutil

DRIVE_ROOT = "/content/drive/My Drive/projects/TensorMorph"
LOCAL_ROOT = "/content/tensormorph_local"

# Sync to local
if os.path.exists(LOCAL_ROOT):
    shutil.rmtree(LOCAL_ROOT)
shutil.copytree(DRIVE_ROOT, LOCAL_ROOT)

os.chdir(LOCAL_ROOT)

# Set permissions
!chmod +x tm-cli
!chmod +x scripts/benchmark.sh

print("\n--- TensorMorph Ready ---")
# This should finally work
!./tm-cli --help

In [ ]:
# @title 3. Build TensorMorph Optimizer
# This uses the CLI to compile the C++ MLIR passes and tools.
!./tm-cli build

In [ ]:
# @title 4. Ingest Model to MLIR
# This command converts a standard model into the TensorMorph MLIR dialect.
# We're passing a flag to create a dummy test model first.
!./tm-cli ingest --model sample_mnist --output test.mlir

In [ ]:
# @title 5. Verify MLIR Artifact
# Inspecting the generated TOSA/TF dialect code
!head -n 50 test.mlir

In [ ]:
# @title 6. Run TensorMorph Optimizer
# This takes the raw TF MLIR and runs our C++ optimization passes
!./tm-cli optimize test.mlir